In [1]:
import os

os.chdir('../')

print(os.getcwd())

/Users/gbemidebe/Documents/LSUCourses/Fall2023/CSC7343/homeworks/Project


In [2]:
import torch
import torch.nn as nn

import random
import numpy as np
import csv
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

In [3]:
import transformers

ModuleNotFoundError: No module named 'transformers'

## Read Data

In [5]:
# fix random seeds
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
    
# load CSV dataset
antigen = []
TCR = []
interaction=[]
with open("./data/data.csv") as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    next(csvReader)
    for row in csvReader:
        antigen.append(row[0])
        TCR.append(row[1])
        interaction.append(row[2])

antigen = np.asarray(antigen)
TCR = np.asarray(TCR)
interaction = np.asarray(interaction, dtype="float")


dataset_size = len(antigen)
all_ind = np.arange(dataset_size)

## Prepare input

In [8]:
# Determine the maximum lengths of antigen and TCR sequences
max_antigen_length = max(len(item[0]) for item in antigen)
max_tcr_length = max(len(item[1]) for item in TCR)

# Function to pad sequences to a given length
def pad_sequence(sequence, max_length):
    return sequence + 'X' * (max_length - len(sequence))  # Using 'X' as the padding character

# Preprocess and create the combined sequences
preprocessed_data = []
for antigen, tcr in zip(antigen, TCR):
    padded_antigen = pad_sequence(antigen, max_antigen_length)
    padded_tcr = pad_sequence(tcr, max_tcr_length)
    combined_sequence = f"<cls>{padded_antigen}<sep>{padded_tcr}"
    preprocessed_data.append(combined_sequence)

# Displaying the first few combined sequences for verification
preprocessed_data[:5]  # Display the first 5 combined sequences


['<cls>AAGIGILTV<sep>CAISEVGVGQPQHF',
 '<cls>AAGIGILTV<sep>CASSLSFGTEAFF',
 '<cls>AARAVFLAL<sep>CASLGAQNNEQF',
 '<cls>AARAVFLAL<sep>CASSYSTGDEQYF',
 '<cls>AIMDKNIIL<sep>CASSVDGGSQPQHF']

In [11]:
preprocessed_data[5:100]

['<cls>ALGIGILTV<sep>CASSFLGTGVEQYF',
 '<cls>ALIHHNTHL<sep>CASSIESGSKQRNEQFF',
 '<cls>ALIHHNTHL<sep>CASSLAVQRPSGNTIYF',
 '<cls>ALIHHNTHL<sep>CASSPRDRVHEQYF',
 '<cls>ALIHHNTHL<sep>CASSQSETGDGYTF',
 '<cls>ALIHHNTYL<sep>CASSDLQENQPQHF',
 '<cls>ALIHHNTYL<sep>CSVVGTGSVITNEKLFF',
 '<cls>ALLETLSLLL<sep>CSARNRDTYYNEQFF',
 '<cls>ALLQVTLLL<sep>CASSSTDGYQETQYF',
 '<cls>ALSPVIPHI<sep>CASSFSGTFEAFF',
 '<cls>ALSPVIPHI<sep>CASSPVLGTYGYTF',
 '<cls>ALSPVIPHI<sep>CASSYFLEQYF',
 '<cls>ALSPVIPLI<sep>CAISETGGGQPQHF',
 '<cls>ALSPVIPLI<sep>CASSGAAGTSAYNEQFF',
 '<cls>ALSPVIPLI<sep>CASSGQGAYITDTQYF',
 '<cls>ALSPVIPLI<sep>CASSLGTLNEQFF',
 '<cls>ALSPVIPLI<sep>CASSPVLGTYGYTF',
 '<cls>ALSPVIPLI<sep>CASSQGPPGSGGGNEQFF',
 '<cls>ALSPVIPLI<sep>CASSVETGGLDTQYF',
 '<cls>ALSPVIPLI<sep>CASSYSSDRSSYEQYF',
 '<cls>ALSPVIPLI<sep>CASYPISRASYEQYF',
 '<cls>ALSPVIPLI<sep>CSATCNGTFLYQETQYF',
 '<cls>ALSYTPAEV<sep>CASSLGQEIPYYGYTF',
 '<cls>ALSYTPAEV<sep>CASSSPSGYEQYF',
 '<cls>ALSYTPAEV<sep>CASSTQGTGVYNEQFF',
 '<cls>ALTPVVVTL<sep>CAS